In [2]:
pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installatio

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'ktrain'

In [ ]:
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df

In [ ]:
data_train, data_test = train_test_split(df, test_size=0.2, random_state=0)
data_train.shape, data_test.shape

## Prepare the data

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train, # using ktrain
                                                                text_column='text',
                                                                label_columns='label',
                                                                val_df = data_test,
                                                                preprocess_mode='bert') # or distilbert

In [ ]:
X_train

In [ ]:
preproc

In [ ]:
len(X_train)

## Build the classifier


In [ ]:
model = text.text_classifier(name='bert',
                             train_data=(X_train, y_train),
                             preproc=preproc)

In [ ]:
type(model) # functional model

In [ ]:
# batch size is taken as 6 as its recommended in the documentation for maxlen=500
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train), val_data=(X_test, y_test), batch_size=6)
learner

In [ ]:
learner.lr_find(show_plot=True, max_epochs=2)

In [ ]:
learner.fit_onecycle(lr=10e-3, epochs=1)
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('bertspam--classifier')


In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)
predictor2 = ktrain.get_predictor(learner.model, preproc)
predictor2.save('bertspamlog2--classifier')

In [ ]:
file = open('sample.csv')
lines = file.readlines()
lines

In [ ]:
predictor.predict(lines[0])

In [ ]:
predictor2.predict(lines[0])

In [1]:
import gradio as gr
def predictor(sent):
    return predictor.predict(sent)

iface = gr.Interface(fn=predictor, inputs="text", outputs="text")
iface.launch()

ModuleNotFoundError: No module named 'gradio'